# Iteration 7: waiting time (KPI)

## Plain English Summary

In this iteration we add a third basic performance measure calculation

3. Mean waiting time for unplanned admissions.

These calculations only use statistics collected in the results collection period.


## Notebook imports

In [1]:
from IPython.display import Markdown, display

import numpy as np
import math

## LLM Prompt

In [2]:
display(Markdown("inputs/iteration_7.md"))

modify the code as follows: 

For unplanned admissions, track the waiting time for critical care bed.  After the patient has taken the bed, if the simulation time has exceeded the warm-up period, the patient's waiting time should be added to a running total and the total number of unplanned admissions should be incremented by 1.

Add in a new performance measure calculation at the end of the simulation run:

* The mean time an unplanned admission had to wait for a critical care bed.  

Add the result to the pandas dataframe.

Show all of the model code: this includes all patient generator functions in the CCUModel class and all code in the Experiment class.



## Date and time of input to LLM

> 1530 3rd December 2023 

## Observations on the LLM Output

* Two new performance measures have been added to `Experiment`

```python
self.mean_waiting_time_unplanned = 0
self.total_unplanned_admissions = 0
```

* Only `unplanned_admission` appears to have been modified.

* Waiting time is tracked correctly for an unplanned patients

```python
arrival_time = self.env.now
    with self.critical_care_beds.request() as req:
        yield req
        wait_time = self.env.now - arrival_time
```

* And stored only when the simulation is in the results collection period

```python
if self.env.now > self.experiment.warm_up_period:
    self.experiment.total_treatment_time += treatment_time
    self.experiment.mean_waiting_time_unplanned += wait_time
    self.experiment.total_unplanned_admissions += 1
```

* Final calculation is correct

```python
mean_waiting_time_unplanned = self.experiment.mean_waiting_time_unplanned / self.experiment.total_unplanned_admissions
```



## LLM Output 1:

In this modified code, the Experiment class stores all results collection variables, including the new performance measure "mean waiting time for unplanned admission". The CCUModel class calculates the total treatment time for both unplanned and elective admissions, and calculates the new performance measure "mean waiting time for unplanned admission" at the end of the simulation run. If the simulation time has exceeded the warm-up period, the code collects statistics on the performance measures and outputs the results in a pandas dataframe.

In [3]:
import simpy
import numpy as np
import pandas as pd

class Experiment:
    def __init__(self, 
                 interarrival_means=[22.72, 26.0, 37.0, 47.2, 575.0, 17.91], 
                 stay_distributions=[(128.79, 267.51), (177.89, 276.54), (140.15, 218.02), (212.86, 457.67), (87.53, 108.67), 57.34], 
                 elective_treatment_mean=57.34, 
                 num_critical_care_beds=24, 
                 intensive_cleaning_duration=5, 
                 warm_up_period=30*24, 
                 results_collection_period=12*30*24):
        self.interarrival_means = interarrival_means
        self.stay_distributions = stay_distributions
        self.elective_treatment_mean = elective_treatment_mean
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.warm_up_period = warm_up_period
        self.results_collection_period = results_collection_period
        self.total_treatment_time = 0
        self.cancelled_elective_count = 0
        self.mean_waiting_time_unplanned = 0
        self.total_unplanned_admissions = 0

class CCUModel:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_count = 0
        self.critical_care_beds = simpy.Resource(env, capacity=experiment.num_critical_care_beds)

    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[0]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[0]))

    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[1]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[1]))

    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[2]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[2]))

    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[3]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[3]))

    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[4]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[4]))

    def patient_arrival_elective_surgery(self):
        while True:
            yield self.env.timeout(np.random.normal(self.experiment.interarrival_means[5], 3.16))
            self.patient_count += 1
            print(f"Elective surgery patient {self.patient_count} arrived at time {self.env.now}")
            if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
                print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
                if self.env.now > self.experiment.warm_up_period:
                    self.experiment.cancelled_elective_count += 1
            else:
                self.env.process(self.elective_surgery_process(self.experiment.elective_treatment_mean))

    def unplanned_admission(self, stay_distribution):
        arrival_time = self.env.now
        with self.critical_care_beds.request() as req:
            yield req
            wait_time = self.env.now - arrival_time
            print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            if self.env.now > self.experiment.warm_up_period:
                self.experiment.total_treatment_time += treatment_time
                self.experiment.mean_waiting_time_unplanned += wait_time
                self.experiment.total_unplanned_admissions += 1
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def elective_surgery_process(self, treatment_mean):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Elective surgery patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.exponential(treatment_mean)
            yield self.env.timeout(treatment_time)
            print(f"Elective surgery patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            if self.env.now > self.experiment.warm_up_period:
                self.experiment.total_treatment_time += treatment_time
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def run(self):
        self.env.process(self.patient_arrival_AE())
        self.env.process(self.patient_arrival_wards())
        self.env.process(self.patient_arrival_surgery())
        self.env.process(self.patient_arrival_other_hospitals())
        self.env.process(self.patient_arrival_X_ray())
        self.env.process(self.patient_arrival_elective_surgery())
        self.env.run(until=self.experiment.results_collection_period + self.experiment.warm_up_period)
        if self.env.now > self.experiment.warm_up_period:
            mean_waiting_time_unplanned = self.experiment.mean_waiting_time_unplanned / self.experiment.total_unplanned_admissions
            bed_utilization = self.experiment.total_treatment_time / (self.experiment.num_critical_care_beds * self.experiment.results_collection_period)
            performance_measures = pd.DataFrame({'Cancelled Elective Operations': [self.experiment.cancelled_elective_count], 'Bed Utilization': [bed_utilization], 'Mean Waiting Time Unplanned': [mean_waiting_time_unplanned]})
            print(performance_measures)

# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the Experiment class and the CCU model, then run the simulation
experiment = Experiment()
ccu_model = CCUModel(env, experiment)
ccu_model.run()


Patient 1 arrived from other hospitals at time 1.2345177980701356
Patient 1 admitted to critical care bed at time 1.2345177980701356
Patient 2 arrived from Accident and Emergency at time 7.43078106579439
Patient 2 admitted to critical care bed at time 7.43078106579439
Patient 3 arrived from other hospitals at time 15.584248227059302
Patient 3 admitted to critical care bed at time 15.584248227059302
Patient 4 arrived from the Wards at time 18.66573977074647
Patient 4 admitted to critical care bed at time 18.66573977074647
Patient 5 arrived from Accident and Emergency at time 18.843137892874342
Patient 5 admitted to critical care bed at time 18.843137892874342
Elective surgery patient 6 arrived at time 22.015762298207825
Elective surgery patient 6 admitted to critical care bed at time 22.015762298207825
Patient 6 discharged from critical care bed at time 23.96395786987179
Patient 6 discharged from critical care bed at time 27.184488808119454
Patient 7 arrived from Accident and Emergency 

## LLM Input 2

## Testing

Here we do a number of regression tests to check that previous functionality is not affected by the `Experiment` class. We also update the test of run length splitting into two tests varying a results collection period and warm-up period.

* **Extreme value test 1**: Wards, Em Surgery, other hospitals, x-ray have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
* **Extreme value test 2**: All unplanned patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Elective."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Elective."
* **Different results collection period**: The results collection period of the model is set to 10 hours.
    * Expected result: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
    * Actual result **(PASS)**: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
* **Extreme value test 3**: `critical_care_beds` = 1
    * Expected result: queues form after first arrival.
    * Actual result **(PASS)**: queues form after first arrival. 
* **Extreme value test 4**: means of stay_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted.
    * Actual result **(PASS)**: after 24 arrivals queues form and no patients are admitted.
* **Vary warm up period**: Vary the warm-up period while holding the results collection period constant.
    * Expected result: Run length is equal to the `results_collection_period`
    * Actual result **(PASS)**: Run length is equal to the `results_collection_period`

In [4]:
# The extreme value
M = 10_000_000

### Test 1: Extreme value test 1

PASS

In [5]:
def run_test(experiment):
    # Create a SimPy environment
    env = simpy.Environment()
    ccu_model = CCUModel(env, experiment)
    ccu_model.run()
    return ccu_model

In [6]:
def extreme_value_test_1(extreme_value=M):
    '''
    Extreme value test 1: 
    
    Wards, Em Surgery, other hospitals, x-ray, electives have their inter-arrival time
    set to $M$ a very large number.
    Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    There are no elective cancellations (as no electives arrive)
    '''
    mod_interarrival_means = [22.72] + [extreme_value] * 5 
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [7]:
extreme_value_test_1()

Patient 1 arrived from Accident and Emergency at time 9.792041631014898
Patient 1 admitted to critical care bed at time 9.792041631014898
Patient 2 arrived from Accident and Emergency at time 11.320280905423953
Patient 2 admitted to critical care bed at time 11.320280905423953
Patient 3 arrived from Accident and Emergency at time 14.82167077194496
Patient 3 admitted to critical care bed at time 14.82167077194496
Patient 3 discharged from critical care bed at time 16.689979498365155
Critical care bed is available for next patient at time 21.689979498365155
Patient 4 arrived from Accident and Emergency at time 34.7228543999881
Patient 4 admitted to critical care bed at time 34.7228543999881
Patient 5 arrived from Accident and Emergency at time 53.87706155727422
Patient 5 admitted to critical care bed at time 53.87706155727422
Patient 6 arrived from Accident and Emergency at time 61.084488563669126
Patient 6 admitted to critical care bed at time 61.084488563669126
Patient 7 arrived from A

### Test 2: Extreme value test 2

PASS

In [8]:
def extreme_value_test_2(extreme_value=M):
    '''
    Extreme value test 2: 
    
    All unplanned admissions have their inter-arrival time is
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    mod_interarrival_means=[extreme_value] * 5 + [17.91]
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [10]:
try:
    extreme_value_test_2()
except ZeroDivisionError:
    print("caught zero division error due to extreme value test")

Elective surgery patient 1 arrived at time 16.841573544302182
Elective surgery patient 1 admitted to critical care bed at time 16.841573544302182
Elective surgery patient 2 arrived at time 37.244242040302865
Elective surgery patient 2 admitted to critical care bed at time 37.244242040302865
Elective surgery patient 3 arrived at time 51.16961432250967
Elective surgery patient 3 admitted to critical care bed at time 51.16961432250967
Elective surgery patient 3 discharged from critical care bed at time 52.43946211645097
Critical care bed is available for next patient at time 57.43946211645097
Elective surgery patient 3 discharged from critical care bed at time 67.0304169850546
Elective surgery patient 4 arrived at time 69.50595882803411
Elective surgery patient 4 admitted to critical care bed at time 69.50595882803411
Critical care bed is available for next patient at time 72.0304169850546
Elective surgery patient 5 arrived at time 84.83735332728762
Elective surgery patient 5 admitted to 

### Test 3: Different run length

PASS

In [11]:
def test_results_collection_period(new_collect_period):
    '''
    Vary the results collection period while holding the 
    warm-up period constant.

    Expected result Run length should be no longer than `new_run_length`
    '''
    experiment = Experiment(results_collection_period=new_collect_period)
    model = run_test(experiment)
    return model.env.now

In [12]:
test_results_collection_period(10)

Elective surgery patient 1 arrived at time 14.10189804641963
Elective surgery patient 1 admitted to critical care bed at time 14.10189804641963
Patient 2 arrived from Emergency surgery at time 23.827633555624832
Patient 2 admitted to critical care bed at time 23.827633555624832
Patient 3 arrived from Emergency surgery at time 26.81082594301072
Patient 3 admitted to critical care bed at time 26.81082594301072
Elective surgery patient 4 arrived at time 30.65564661604381
Elective surgery patient 4 admitted to critical care bed at time 30.65564661604381
Elective surgery patient 4 discharged from critical care bed at time 32.01303719280279
Critical care bed is available for next patient at time 37.01303719280279
Patient 5 arrived from the Wards at time 41.05725783184602
Patient 5 admitted to critical care bed at time 41.05725783184602
Patient 6 arrived from other hospitals at time 42.67167643405306
Patient 6 admitted to critical care bed at time 42.67167643405306
Patient 6 discharged from c

730

### Test 4: Extreme value test 3

PASS.

In [13]:
def extreme_value_test_3(critical_care_beds=1):
    '''
    Extreme value test 3: 
    
    Critical care beds set to 1 or parameter
    
    Expected result: when critical_care_beds=1 queues form after first arrival.
    Cancellations also begin after 1st arrival.

    Note: reported utilisation will not quite be 100% because of the patient still
    in service.
    
    '''
    experiment = Experiment(num_critical_care_beds=critical_care_beds)
    run_test(experiment)
    print(f'{experiment.total_treatment_time=}')
    print(f'{experiment.results_collection_period=}')

In [14]:
extreme_value_test_3()

Patient 1 arrived from Accident and Emergency at time 2.4899169386156945
Patient 1 admitted to critical care bed at time 2.4899169386156945
Patient 2 arrived from other hospitals at time 8.6326145830335
Elective surgery patient 3 arrived at time 14.317484185447793
Elective surgery for patient 3 cancelled due to no available critical care beds at time 14.317484185447793
Patient 4 arrived from Accident and Emergency at time 23.986275654995623
Elective surgery patient 5 arrived at time 37.29708081977463
Elective surgery for patient 5 cancelled due to no available critical care beds at time 37.29708081977463
Patient 6 arrived from Accident and Emergency at time 40.01242372541213
Patient 7 arrived from other hospitals at time 55.848174996305005
Elective surgery patient 8 arrived at time 56.12484424907797
Elective surgery for patient 8 cancelled due to no available critical care beds at time 56.12484424907797
Patient 9 arrived from Accident and Emergency at time 58.3842603264482
Patient 10 a

### Test 5: Extreme value test 4

PASS

In [15]:
def extreme_value_test_4(extreme_value=M):
    '''
    Extreme value test 4: 
    
    means of stay_distributions set to $M$ a very large number.
    Expected result: after 24 arrivals queues form and no patients are admitted.
    '''
    mod_stay_distributions=[(M, 267.51), 
                            (M, 276.54), 
                            (M, 218.02), 
                            (M, 457.67), 
                            (M, 108.67), 
                            57.34] 
    experiment = Experiment(stay_distributions=mod_stay_distributions, 
                            elective_treatment_mean=M)
    run_test(experiment)

In [18]:
try:
    extreme_value_test_4()
except ZeroDivisionError:
    print("caught expected zero division error due to extreme value test")

Patient 1 arrived from the Wards at time 1.955820203508696
Patient 1 admitted to critical care bed at time 1.955820203508696
Patient 2 arrived from Accident and Emergency at time 3.9858391632974577
Patient 2 admitted to critical care bed at time 3.9858391632974577
Patient 3 arrived from Accident and Emergency at time 6.106092912909711
Patient 3 admitted to critical care bed at time 6.106092912909711
Patient 4 arrived from the Wards at time 8.041790329952374
Patient 4 admitted to critical care bed at time 8.041790329952374
Patient 5 arrived from Emergency surgery at time 8.71775232787417
Patient 5 admitted to critical care bed at time 8.71775232787417
Patient 6 arrived from the Wards at time 12.54758105907439
Patient 6 admitted to critical care bed at time 12.54758105907439
Elective surgery patient 7 arrived at time 14.620474525780082
Elective surgery patient 7 admitted to critical care bed at time 14.620474525780082
Patient 8 arrived from the Wards at time 14.874943124652269
Patient 8 

### Test 6: Vary Warm-up parameter

PASS

In [19]:
def test_warmup_period(new_warmup_period):
    '''
    Vary the warm-up period while holding the 
    results collection period constant.

    Expected result Run length should be no longer than results_collection_period
    '''
    experiment = Experiment(warm_up_period=new_warmup_period)
    model = run_test(experiment)
    return model.env.now, experiment.results_collection_period

In [20]:
test_warmup_period(0)

Patient 1 arrived from other hospitals at time 1.3688203876540102
Patient 1 admitted to critical care bed at time 1.3688203876540102
Patient 2 arrived from Accident and Emergency at time 1.3784149174740785
Patient 2 admitted to critical care bed at time 1.3784149174740785
Patient 3 arrived from Accident and Emergency at time 12.53496923171578
Patient 3 admitted to critical care bed at time 12.53496923171578
Elective surgery patient 4 arrived at time 19.277771158627942
Elective surgery patient 4 admitted to critical care bed at time 19.277771158627942
Patient 5 arrived from Emergency surgery at time 20.281885654910642
Patient 5 admitted to critical care bed at time 20.281885654910642
Patient 6 arrived from Accident and Emergency at time 26.94483936963832
Patient 6 admitted to critical care bed at time 26.94483936963832
Elective surgery patient 7 arrived at time 32.50281210978307
Elective surgery patient 7 admitted to critical care bed at time 32.50281210978307
Patient 8 arrived from oth

(8640, 8640)